# gonna use XGB now to combine the categorical, continuous and pre-embedded free text stuff

#### Online, we import the usual packages. **xgboost** needs to be installed (with conda install xgboost or pip install -U xgboost).

In [29]:
#!pip install xgboost

####  the usual imports

In [2]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics
from collections import Counter
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import random as rn
from IPython.display import clear_output
import pickle

### load helper functions

In [3]:
def thresholding_analysis(preds, targets, admission_thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]):

    i = 0
    for thresh in admission_thresholds:
        thresholded_predictions = [0 if prob[0] >= thresh else 1 for prob in preds]
        
        f1_w, f1, acc, prec, rec, auroc = get_metrics(targets,thresholded_predictions, print_output = False)

        if i == 0:
            output_df = pd.DataFrame([thresh, f1_w, f1, acc, prec, rec, rec[0], rec[1]]).T
            output_df.columns = ['admission_threshold', 'AUROC', 'f1', 'accuracy', 'precision', 'recall', 'admission sensitivity', 'admission specificity']
        else:
            output_df.loc[len(output_df)] = [thresh, auroc, f1, acc, prec, rec, rec[0], rec[1]]
        i+=1
    
    return output_df

In [4]:

def show_confusion_matrix2(confusion_matrix, labels):
    LABELS = labels

    plt.figure(figsize=(3, 3))
    sns.heatmap(confusion_matrix, cmap = 'Blues', xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", annot_kws={"size": 15});
    plt.title("Confusion matrix", fontsize=10)
    plt.ylabel('True label', fontsize=10)
    plt.xlabel('Predicted label', fontsize=10)
    plt.show()
    
def get_metrics(model, y_test, X_test, show_confusion = True, print_output = True, return_values = False):
    preds = model.predict_proba(X_test)
    predictions = model.predict(X_test)
    
    f1_w = sklearn.metrics.f1_score(y_test, predictions, average='weighted')
    f1 = sklearn.metrics.f1_score(y_test, predictions, average=None)
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    prec = sklearn.metrics.precision_score(y_test,predictions, average=None) 
    rec = sklearn.metrics.recall_score(y_test,predictions, average=None)
    auroc = sklearn.metrics.roc_auc_score(y_test,preds[:,1], average=None)
    confusion = sklearn.metrics.confusion_matrix(y_test, predictions)
    
    tp, fn, fp, tn = confusion[0,0], confusion[0,1], confusion[1,0], confusion[1,1]
    
    sens = tp/(tp + fn)
    spec = tn/(tn + fp)
    ppv = tp/(tp + fp)
    npv = tn/(tn + fn)
    
    
    if print_output:
        print ('Metrics Report:')
        print ('---------------')
        print ('weighted f1: ', f1_w)
        print ('AUROC:       ',auroc)
        print ('accuracy:    ', acc)
        print ('precision:   ', prec)
        print ('recall:      ', rec)
        #print ('admission sens: ', rec[0])
        #print ('admission spec: ', rec[1])
        print ('sensitivity: ', sens)
        print ('specificity: ', spec)
        print ('PPV:         ', ppv)
        print ('NPV:         ', npv)
        
    if show_confusion:
        show_confusion_matrix2(confusion, labels = ['admit', 'd/c'])
     
    if return_values == True:
        return f1_w, auroc, acc, sens, ppv




In [5]:
def JJ_gridsearch(weights, print_output = False):
    epoch = 0
    for weight in weights:
        
        xgc = xgb.XGBClassifier(scale_pos_weight=weight)
        xgc.fit(X_train, y_train)
        preds = xgc.predict(X_test)
        probas = xgc.predict_proba(X_test)
        f1_w, f1, acc, prec, rec = get_metrics(y_test, preds, print_output)
        
        if epoch == 0:
            results_df = pd.DataFrame([epoch+1, 1/weight, f1_w, f1, acc, prec, rec, rec[0], rec[1]]).T
            results_df.columns = ['trial number', 'class penalty', 'weighted f1', 'f1', 'accuracy', 'precision', 'recall', 'admission sensitivity', 'admission specificity']
        else:
            results_df.loc[len(results_df)] = [epoch+1, 1/weight, f1_w, f1, acc, prec, rec, rec[0], rec[1]]
        
        epoch +=1
        
    return xgc, results_df, preds, probas

In [6]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'
model_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage/models'

In [7]:
clin = pd.read_csv(data_path + '/complete_clean_combo_data.csv', index_col = 0,low_memory = False)

In [8]:
dx_code_dict = {code:i for i,code in enumerate(set(clin['MainDiagnosisCode']))}

def convert_dxcode(s):
    code = dx_code_dict[s]
    return (code)

clin['recoded_diagnosis'] = clin['MainDiagnosisCode'].map(convert_dxcode)

In [9]:
medhx = np.load(data_path + '/medhx_embeds.npy')

#this is the admit vs d/c target
target = np.load(data_path + '/admit_dc_target.npy')

subjnotes = np.load(data_path + '/subj_emeds.npy')


In [10]:
bch_data = clin[clin['site']== 'BCH']

In [11]:
bch_idxs = np.array(list(bch_data.index)); len(bch_idxs)

102128

In [12]:
medhx = medhx[bch_idxs]
subjnotes = subjnotes[bch_idxs]
target = target[bch_idxs]

In [13]:
clin = bch_data

In [14]:
#this cell will define the various groupings of variables from the dataframe
dx_vars = ['recoded_diagnosis', 'PresentingComplaint']

cont_vars = [ 'Triage Date & TimeYear', 'Triage Date & TimeMonth', 'Triage Date & TimeWeek', 'Triage Date & TimeDay',
 'Triage Date & TimeDayofweek', 'Triage Date & TimeDayofyear', 'Triage Date & TimeHour', 'Triage Date & TimeMinute',
 'Triage Date & TimeSecond', 'Triage Date & TimeElapsed',
 'num_comorbids','systolic', 'diastolic', 'o2sat', 'pulse', 'temp', 'AgeInYrs']


cos_date_vars = ['Triage Date & Timeweekday_cos',
       'Triage Date & Timeweekday_sin', 'Triage Date & Timeday_month_cos',
       'Triage Date & Timeday_month_sin', 'Triage Date & Timemonth_year_cos',
       'Triage Date & Timemonth_year_sin', 'Triage Date & Timeday_year_cos',
       'Triage Date & Timeday_year_sin', 'Triage Date & Timehour_cos',
       'Triage Date & Timehour_sin', 'Triage Date & Timeclock_cos',
       'Triage Date & Timeclock_sin', 'Triage Date & Timemin_cos',
       'Triage Date & Timemin_sin', 'Triage Date & Timesec_cos']

cat_vars = ['Triage Date & TimeIs_month_end',
 'Triage Date & TimeIs_month_start',
 'Triage Date & TimeIs_quarter_end',
 'Triage Date & TimeIs_quarter_start',
 'Triage Date & TimeIs_year_end',
 'Triage Date & TimeIs_year_start',
 'GenderDesc', 'TriageLevel', 'site']

inf_control_vars = ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 'Do you have a new Rash?',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 'Have you received Health Care in another country in the last 2 years?',
 'Do you have a new/worse cough or shortness of breath?',
 'If so, select all countries that apply',
 'If so, select all infectious diseases that apply']

### preprocessing variables to use in XGB

In [15]:
#need to preprocess cat vars for xgb
X = clin[cat_vars + inf_control_vars].values.astype(str)

features = []
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    features.append(feature)
encoded_x = np.array(features)
encoded_x = encoded_x.reshape(X.shape[0], X.shape[1])

In [16]:
encoded_x.shape

(102128, 19)

In [17]:
X2 = clin[dx_vars].values.astype(str)
encoded_x2 = None
for i in range(0, X2.shape[1]):
	label_encoder = LabelEncoder()
	feature = label_encoder.fit_transform(X2[:,i])
	feature = feature.reshape(X2.shape[0], 1)
	onehot_encoder = OneHotEncoder(sparse=False)
	feature = onehot_encoder.fit_transform(feature)
	if encoded_x2 is None:
		encoded_x2 = feature
	else:
		encoded_x2 = np.concatenate((encoded_x2, feature), axis=1)
print("X shape: : ", encoded_x2.shape)

X shape: :  (102128, 3292)


In [18]:
#splitting this one hot encoded matrix into one for the presenting complaint and one for the medical history
X_pres = encoded_x2[:,:169]

X_dx = encoded_x2[:,169:]

In [19]:
features = np.concatenate((encoded_x,X_pres, X_dx, clin[cont_vars].values), axis =1)

all_features = np.concatenate((features,subjnotes,medhx), axis = 1)
all_features.shape

(102128, 4864)

### dataset size analysis

In [25]:
       
dataset_sizes = [10000, 30000, 60000, 100000, 130000, len(clin)]    

#initialize dataframe for results
col_names = ['num_examples','train_f1', 'test_f1', 'train_acc', 'test_acc', 'train_auroc', 'test_auroc', 'train_ppv', 'test_ppv', 'train_sens', 'test_sens']
output_df = pd.DataFrame(columns = col_names)

for size in dataset_sizes:
    print ()
    print ('Training with {} examples'.format(size))
    print ()
    
    X_train, X_test, y_train, y_test = train_test_split(all_features[:size], target[:size], random_state=1)
    
    xgc = xgb.XGBClassifier(scale_pos_weight = 1/9, gamma = 0.5, reg_alpha = 0.5)
    %time xgc.fit(X_train, y_train)
    
    #random train subset for metrics
    random_indices = np.random.choice(X_test.shape[0], len(X_test), replace=False)
    
    print ('\nTraining Set')
    train_f1, train_auroc, train_acc, train_sens, train_ppv = get_metrics(xgc, y_train[random_indices], X_train[random_indices],
                                                                          show_confusion = False, return_values = True)
    print ('\nValidation Set')
    test_f1, test_auroc, test_acc, test_sens, test_ppv = get_metrics(xgc, y_test, X_test,
                                                                     show_confusion = False, return_values = True)
    
    output_df.loc[len(output_df)] = [size,train_f1, test_f1, train_acc, test_acc, train_auroc, test_auroc, train_ppv, test_ppv, train_sens, test_sens]


Training with 10000 examples

CPU times: user 1min 28s, sys: 707 ms, total: 1min 29s
Wall time: 1min 32s

Training Set


/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Metrics Report:
---------------
weighted f1:  0.8757229107398223
AUROC:        0.956074476450177
accuracy:     0.852
precision:    [0.39160839 0.98858921]
recall:       [0.91056911 0.84560781]
sensitivity:  0.9105691056910569
specificity:  0.8456078083407276
PPV:          0.3916083916083916
NPV:          0.9885892116182573

Validation Set
Metrics Report:
---------------
weighted f1:  0.8351382652600045
AUROC:        0.8336372042029591
accuracy:     0.8064
precision:    [0.2962298  0.95265054]
recall:       [0.64202335 0.82523406]
sensitivity:  0.642023346303502
specificity:  0.8252340615247437
PPV:          0.296229802513465
NPV:          0.952650540401441

Training with 30000 examples

CPU times: user 4min 20s, sys: 1.52 s, total: 4min 21s
Wall time: 4min 24s

Training Set


/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Metrics Report:
---------------
weighted f1:  0.840994321103802
AUROC:        0.8916282892186508
accuracy:     0.8102666666666667
precision:    [0.32690247 0.97035854]
recall:       [0.78507079 0.81317864]
sensitivity:  0.7850707850707851
specificity:  0.8131786404878775
PPV:          0.32690246516613075
NPV:          0.9703585374511892

Validation Set


...

Metrics Report:
---------------
weighted f1:  0.8375575693926557
AUROC:        0.8605533089686239
accuracy:     0.8022481591728028
precision:    [0.27996087 0.96736945]
recall:       [0.7306383  0.80950738]
sensitivity:  0.7306382978723405
specificity:  0.8095073764127341
PPV:          0.2799608674384477
NPV:          0.96736945203361

Training with 102128 examples

CPU times: user 15min 20s, sys: 11.9 s, total: 15min 32s
Wall time: 15min 40s

Training Set


/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Metrics Report:
---------------
weighted f1:  0.8383296292918467
AUROC:        0.8669219662208305
accuracy:     0.8047939840200533
precision:    [0.29895917 0.96857987]
recall:       [0.7549535  0.81013921]
sensitivity:  0.7549534977759806
specificity:  0.8101392081183052
PPV:          0.29895916733386707
NPV:          0.9685798724529476

Validation Set


/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Metrics Report:
---------------
weighted f1:  0.8375575693926557
AUROC:        0.8605533089686239
accuracy:     0.8022481591728028
precision:    [0.27996087 0.96736945]
recall:       [0.7306383  0.80950738]
sensitivity:  0.7306382978723405
specificity:  0.8095073764127341
PPV:          0.2799608674384477
NPV:          0.96736945203361


In [26]:
output_df

,num_examples,train_f1,test_f1,train_acc,test_acc,train_auroc,test_auroc,train_ppv,test_ppv,train_sens,test_sens
0,10000.0,0.875723,0.835138,0.852000,0.806400,0.956074,0.833637,0.391608,0.296230,0.910569,0.642023
1,30000.0,0.840994,0.823505,0.810267,0.784800,0.891628,0.839609,0.326902,0.267255,0.785071,0.718271
2,60000.0,0.838752,0.829794,0.804200,0.794533,0.876428,0.858988,0.295405,0.288512,0.768096,0.734501
3,100000.0,0.840489,0.838140,0.807000,0.804320,0.870252,0.852728,0.295859,0.284480,0.750633,0.715077
4,130000.0,0.838330,0.837558,0.804794,0.802248,0.866922,0.860553,0.298959,0.279961,0.754953,0.730638
5,102128.0,0.838330,0.837558,0.804794,0.802248,0.866922,0.860553,0.298959,0.279961,0.754953,0.730638


In [27]:
cols = list(clin.columns)

In [28]:
cols

['ID',
 'ChartNumber',
 'EncounterNumber',
 'TriageLevel',
 'AgeNumber',
 'AgeInYrs',
 'GenderDesc',
 'PIA Date & Time',
 'Disposition Date & Time',
 'DischargeDisposition',
 'DischargeDispositionDesc',
 'Left ED Date & Time',
 'PresentingComplaint',
 'PresentingComplaintDesc',
 'MainDiagnosisCode',
 'MainDiagnosisCodeDesc',
 'AdmitLocation',
 'PatientService',
 'SubjectiveNotes',
 'InfectionControlScreening',
 'MedicalHistory',
 'BloodPressure_LastEDReading',
 'O2Saturation_LastEDReading',
 'Pulse_LastEDReading',
 'Temperature_LastEDReading',
 'o2sat',
 'pulse',
 'temp',
 'CleanSubjectiveNotes',
 'BP',
 'systolic',
 'diastolic',
 'Gender',
 'outcome',
 'target',
 'service',
 'pmhx',
 'num_comorbids',
 'target2',
 'discharge',
 'target3',
 'dispo',
 'target4',
 'ICUvsother',
 'target5',
 'CleanMedicalHistory',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 'Have you ever been isolated/required isolation for an infectious disease when receiving care in 

### augmentation

In [21]:
Counter(clin['target3'])

Counter({1.0: 92436, 0.0: 9692})

In [22]:
Counter(target)

Counter({1.0: 92436, 0.0: 9692})

In [23]:
Counter(clin['discharge'])

Counter({'discharge': 92436, 'admit': 9692})

In [24]:
admits = clin[clin['discharge'] == 'admit']

In [25]:
admits.shape

(9692, 122)